# YIELDS.IO | Stress Testing with GANs 
---------------------
Main functions to use in order to setup and train a GAN model

---------------
- **Step 1:** Import libraries 
- **Step 2:** Implement a standalone Discriminator NN
- **Step 3:** Implement a standalone Generator NN 
- **Step 4:** Implement a composite GAN NN
- **Step 5:** Train the GAN NN 
- **Step 6:** Evaluate Performance
-----------------
#### Additional information
- Overview of GANs: [GAN Google](https://developers.google.com/machine-learning/gan/summary)
- Seminal GAN paper of Goodfellow et al.: [GAN Seminal paper](https://arxiv.org/abs/1406.2661)
- Instability problems with GANs: [Stabilizing GANs](https://arxiv.org/pdf/1910.00927.pdf)
- Wasserstein GAN to deal with vanishing gradients: [WGAN](https://arxiv.org/abs/1701.07875) 
- Improved WGAN with Gradient Penalty: [WGAN-GP](https://arxiv.org/abs/1704.00028)
- How to add custom gradient penalty in Keras: [WGAN and Keras](https://keras.io/examples/generative/wgan_gp/) and [Change model.fit()](https://keras.io/guides/customizing_what_happens_in_fit/)
- Multi-categorical variables and GAN: [Multi-categorical GAN](https://arxiv.org/pdf/1807.01202.pdf)
- Alternative to WGAN: [DRAGAN](https://arxiv.org/abs/1705.07215)
- Conditional DRAGAN on American Express dataset: [DRAGAN & American Express](https://arxiv.org/pdf/2002.02271.pdf)
- Mulitvariate KL-divergence by KNN: [KL by KNN](https://ieeexplore.ieee.org/document/4035959)
- Multivariate KL-divergence by  KNN p2: [KL by KNN](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.422.5121&rep=rep1&type=pdf)

## Import Libraries  
----------

In [ ]:
# Basic libraries 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%config Completer.use_jedi = False #to increase speed of tab autocompletion (remove if obsolete/redundant)

# Keras modules 
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from keras.layers import Dense
from keras.layers import LeakyReLU, ReLU
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras import layers
import keras

# Tensorflow (for WGAN-GP Gradient Penalty)
import tensorflow as tf

# KNN computation used by KL-divergence function 
from scipy.spatial import cKDTree


# DISCRIMINATOR: Set of Discriminators to use with GAN 
--------------------------------------------------------
- Most relevant version as of 26/08/2020: **discriminator_setup_GENERIC**
- All other versions are redundant and only used for experiments

In [ ]:
def discriminator_setup_basic(input_shape, optimizer= Adam(lr=0.0002, beta_1=0.5)):
    '''
    #input_shape: the dimension of the dataset we need to classify as real or fake. 
    ## Works well with nominal data and MV Gaussian
    '''

    # Model setup (X's are hidden layers)
    inputs= layers.Input((input_shape,))
    X= layers.Dense(512, activation= LeakyReLU(alpha=0.2))(inputs)
    X= layers.Dense(256, activation= LeakyReLU(alpha=0.2))(X)
    outputs= layers.Dense(1, activation='sigmoid')
    
    # Compile 
    model= Model(inputs, outputs)
    model.compile(optimizer= optimizer, loss= 'binary_crossentropy', metrics= ['accuracy'])
    
    return model

In [ ]:
def discriminator_setup_CTGAN(input_shape, optimizer= Adam(lr=0.0002, beta_1=0.5)):
    '''
    #input_shape: the dimension of the dataset we need to classify as real or fake. 
    ## Experimental Design CTGAN paper
    '''
    
    # Model setup
    input_layer = layers.Input((input_shape,))
    X= layers.Dense(256, activation=LeakyReLU(alpha= 0.2))(input_layer)
    X= Dropout(0.2)(X)
    X= layers.Dense(256, activation=LeakyReLU(alpha= 0.2))(X)
    X= Dropout(0.2)(X)
    output_layer = layers.Dense(1, activation='sigmoid')(X)
    
    # Compile 
    model = Model(input_layer, output_layer)
    model.compile(optimizer= optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    
    return model
    

In [ ]:
def discriminator_setup_WGANGP(input_shape):
    '''
    #input_shape: the dimension of the dataset we need to classify as real or fake. 
    ## Experimental Design WGANGP paper
    '''
    
    input_layer = layers.Input((input_shape,))
    # Batchnormalizarion is NOT allowed (see paper for alternative if needed)
    X= layers.Dense(256, activation=LeakyReLU(alpha= 0.2))(input_layer)
    X= Dropout(0.2)(X)
    X= layers.Dense(256, activation=LeakyReLU(alpha= 0.2))(X)
    X= Dropout(0.2)(X)
    # uses a linear activation function
    output_layer = layers.Dense(1)(X)
    model = Model(input_layer, output_layer)
    
    # no need to compile (will be done within WGAN class [see below])
    return model

In [ ]:
def discriminator_setup_GENERIC(input_shape, numberOfNodes= 256, numberOfLayers= 2, activationFunction= 'linear'):
    '''
    #input_shape: the dimension of the dataset we need to classify as real or fake. 
    ## Architecture is determined by the end user 
    ## Basic setup is derived from WGAN-GP paper
    '''
    
    input_layer = layers.Input((input_shape,))
    X= input_layer
    # Batchnormalizarion is NOT allowed (see paper for alternative if needed)
    for eachLayer in range(numberOfLayers):
        X= layers.Dense(numberOfNodes, activation= LeakyReLU(alpha= 0.2))(X)
        X= Dropout(0.2)(X)
    # WGAN-GP uses a linear activation function
    output_layer = layers.Dense(1)(X)
    model = Model(input_layer, output_layer)
    
    # no need to compile (will be done within WGAN class [see below])
    return model


# GENERATOR: Set of Generators to use with GAN 
--------------------------------------------------------
- Most relevant version as of 26/08/2020: **generator_setup_WGANGP_GENERIC**
- All other versions are redundant and only used for experiments

In [ ]:
def generator_setup_basic(output_shape, latent_dim, activation= 'tanh'):
    '''
    #output_shape: the dimension of the dataset we need to generate. 
    #laten_dim: the dimension of the latent space where we draw our input for the Generator 
    #activation: the activation function used in the output layer
    ## Works well with nominal data and MV Gaussian
    '''

    # Model setup 
    inputs= layers.Input((latent_dim,))
    X= layers.Dense(256, activation= LeakyReLU(alpha=0.2))
    X= BatchNormalization(momentum=0.8)(X)
    X= layers.Dense(512, activation= LeakyReLU(alpha=0.2))
    X= BatchNormalization(momentum=0.8)(X)    
    X= layers.Dense(1024, activation= LeakyReLU(alpha=0.2))
    X= BatchNormalization(momentum=0.8)(X) 
    outputs= layers.Dense(output_shape, activation= activation)
    
    # No need to compile model -> will be done within composite model 
    model = Model(inputs, outputs )
    return model

In [ ]:
def generator_setup_mixed(output_shape, latent_dim, activation_continous='tanh', activation_discrete='softmax'):
    '''
    #output_shape: the dimension of the dataset we need to generate. 
    #laten_dim: the dimension of the latent space where we draw our input for the Generator 
    #activation_continuous: the activation function used in the output layers connected to continuous variables
    #activation_nominal: the activation function used in the output layers connected to nominal variables
    ## Works well mixed distributions 
    '''
    
    ## Main Model ## 
    inputs= keras.Input(shape=latent_dim)
    X= layers.Dense(256, activation= LeakyReLU(alpha=0.2), )(inputs)
    X= BatchNormalization(momentum=0.8)(X)
    X= layers.Dense(512, activation= LeakyReLU(alpha=0.2),)(X)
    X= BatchNormalization(momentum=0.8)(X)
    X= layers.Dense(1024, activation= LeakyReLU(alpha=0.2),)(X)
    X= BatchNormalization(momentum=0.8)(X)
    outputs= layers.Dense(output_shape)(X) 
    
    
    
    ## Create Dense layers for every column of the dataset 
    # Dense layers 
    continuousColumn= layers.Dense(1, name="continuous1")(outputs)
    discreteColumn= layers.Dense(3, name='discrete1')(outputs)
    
    ## Create new output layers for every column of the dataset 
    continuousColumn= layers.Dense(1, activation= activation_continous, name="continuous2")(continuousColumn)
    discreteColumn= layers.Dense(3, activation= activation_discrete, name='discrete2')(discreteColumn)
    
    ## Concatenate all output layers together again 
    lastLayer= layers.concatenate([continuousColumn, discreteColumn])
    modelFinal= keras.Model(inputs=inputs, outputs=lastLayer)
    return modelFinal

In [ ]:
def generator_setup_ppnr(output_shape, latent_dim, activation_continous='tanh', activation_discrete='softmax'):
    '''
    #output_shape: the dimension of the dataset we need to generate. 
    #laten_dim: the dimension of the latent space where we draw our input for the Generator 
    #activation_continuous: the activation function used in the output layers connected to continuous variables
    #activation_nominal: the activation function used in the output layers connected to nominal variables
    ## Works well PPNR distributions 
    '''
   
    ## Main Model ##
    inputs= keras.Input(shape=latent_dim)
    X= layers.Dense(256, activation= LeakyReLU(alpha=0.2), )(inputs)
    X= BatchNormalization(momentum=0.8)(X)
    X= layers.Dense(512, activation= LeakyReLU(alpha=0.2),)(X)
    X= BatchNormalization(momentum=0.8)(X)
    X= layers.Dense(1024, activation= LeakyReLU(alpha=0.2),)(X)
    X= BatchNormalization(momentum=0.8)(X)
    outputs= layers.Dense(output_shape)(X) 
    
    
    
    
    ## Attach dense layers for every column of the dataset at the output of the main model in parallel 
    continuousColumns= layers.Dense(14, name="continuousDense")(outputs)
    education= layers.Dense(7, name='educationDense')(outputs)
    marriage= layers.Dense(4, name='marriageDense')(outputs)
    pay_1= layers.Dense(11, name='pay1Dense')(outputs)
    pay_2= layers.Dense(10, name='pay2Dense')(outputs)
    pay_3= layers.Dense(11, name='pay3Dense')(outputs)
    pay_4= layers.Dense(11, name='pay4Dense')(outputs)
    pay_5= layers.Dense(10, name='pay5Dense')(outputs)
    pay_6= layers.Dense(10, name='pay6Dense')(outputs)
    sex= layers.Dense(2, name='sexDense')(outputs)
    default_payment_next_month= layers.Dense(2, name='defaultDense')(outputs)
    
    ## Create new output layers for every column of the dataset 
    continuousColumns= layers.Dense(14, activation= activation_continous, name="continuousOutput")(continuousColumns)
    education= layers.Dense(7, activation= activation_discrete, name='educationOutput')(education)
    marriage= layers.Dense(4, activation= activation_discrete, name='marriageOutput')(marriage)
    pay_1= layers.Dense(11, activation= activation_discrete, name='pay1Output')(pay_1)
    pay_2= layers.Dense(10, activation= activation_discrete, name='pay2Output')(pay_2)
    pay_3= layers.Dense(11, activation= activation_discrete, name='pay3Output')(pay_3)
    pay_4= layers.Dense(11, activation= activation_discrete, name='pay4Output')(pay_4)
    pay_5= layers.Dense(10, activation= activation_discrete, name='pay5Output')(pay_5)
    pay_6= layers.Dense(10, activation= activation_discrete, name='pay6Output')(pay_6)
    sex= layers.Dense(2, activation= activation_discrete, name='sexOutput')(sex)
    default_payment_next_month= layers.Dense(2, activation= activation_discrete, name='defaultOutput')(default_payment_next_month)
    
    
    ## Concatenate all output layers together again 
    lastLayer= layers.concatenate([continuousColumns, education, marriage, pay_1, pay_2, pay_3,
                                  pay_4, pay_5, pay_6, sex, default_payment_next_month])
    modelFinal= keras.Model(inputs=inputs, outputs=lastLayer)
    
    
    return modelFinal

In [ ]:
def generator_setup_CTGAN(output_shape, latent_dim, activation_continous='tanh', activation_discrete='softmax'):
    '''
    #output_shape: the dimension of the dataset we need to generate. 
    #laten_dim: the dimension of the latent space where we draw our input for the Generator 
    #activation_continuous: the activation function used in the output layers connected to continuous variables
    #activation_nominal: the activation function used in the output layers connected to nominal variables
    ## Experimental Design of CTGAN paper 
    '''
    
    ## Main Model ## 
    inputs= keras.Input(shape=latent_dim)
    X= layers.Dense(256, activation= ReLU())(inputs)
    X= BatchNormalization()(X)
    X= layers.Dense(256, activation= ReLU())(inputs)
    X= BatchNormalization()(X)
    outputs= layers.Dense(output_shape)(X)
    
    
    ## Create Dense layers for every column of the dataset  
    continuousColumns= layers.Dense(14, name="continuousDense")(outputs)
    education= layers.Dense(7, name='educationDense')(outputs)
    marriage= layers.Dense(4, name='marriageDense')(outputs)
    pay_1= layers.Dense(11, name='pay1Dense')(outputs)
    pay_2= layers.Dense(10, name='pay2Dense')(outputs)
    pay_3= layers.Dense(11, name='pay3Dense')(outputs)
    pay_4= layers.Dense(11, name='pay4Dense')(outputs)
    pay_5= layers.Dense(10, name='pay5Dense')(outputs)
    pay_6= layers.Dense(10, name='pay6Dense')(outputs)
    sex= layers.Dense(2, name='sexDense')(outputs)
    default_payment_next_month= layers.Dense(2, name='defaultDense')(outputs)
    
    ## Create new output layers for every column of the dataset 
    continuousColumns= layers.Dense(14, activation= activation_continous, name="continuousOutput")(continuousColumns)
    education= layers.Dense(7, activation= activation_discrete, name='educationOutput')(education)
    marriage= layers.Dense(4, activation= activation_discrete, name='marriageOutput')(marriage)
    pay_1= layers.Dense(11, activation= activation_discrete, name='pay1Output')(pay_1)
    pay_2= layers.Dense(10, activation= activation_discrete, name='pay2Output')(pay_2)
    pay_3= layers.Dense(11, activation= activation_discrete, name='pay3Output')(pay_3)
    pay_4= layers.Dense(11, activation= activation_discrete, name='pay4Output')(pay_4)
    pay_5= layers.Dense(10, activation= activation_discrete, name='pay5Output')(pay_5)
    pay_6= layers.Dense(10, activation= activation_discrete, name='pay6Output')(pay_6)
    sex= layers.Dense(2, activation= activation_discrete, name='sexOutput')(sex)
    default_payment_next_month= layers.Dense(2, activation= activation_discrete, name='defaultOutput')(default_payment_next_month)
    
    
    ## Concatenate all output layers together again 
    lastLayer= layers.concatenate([continuousColumns, education, marriage, pay_1, pay_2, pay_3,
                                  pay_4, pay_5, pay_6, sex, default_payment_next_month])
    modelFinal= keras.Model(inputs=inputs, outputs=lastLayer)
    return modelFinal

In [ ]:
def generator_setup_WGANGP(output_shape, latent_dim, activation='tanh'):
    '''
    #output_shape: the dimension of the dataset we need to generate. 
    #laten_dim: the dimension of the latent space where we draw our input for the Generator 
    #activation_continuous: the activation function used in the output layers connected to continuous variables
    #activation_nominal: the activation function used in the output layers connected to nominal variables
    ## Experimental Design WGANGP Paper
    '''
     
    inputs= keras.Input(shape=latent_dim)
    X= layers.Dense(256, activation= ReLU())(inputs)
    X= BatchNormalization()(X)
    X= layers.Dense(256, activation= ReLU())(X)
    X= BatchNormalization()(X)
    # Best results are with 'tanh' function to output between [-1,1]
    outputs= layers.Dense(output_shape, activation= activation)(X)
    
    # no need to compile (will be done within WGAN class [see below])
    return keras.Model(inputs, outputs)

In [ ]:
def generator_setup_WGANGP_PPNR(output_shape, latent_dim, activation_continous='tanh', activation_discrete='softmax'):
    '''
    #output_shape: the dimension of the dataset we need to generate. 
    #laten_dim: the dimension of the latent space where we draw our input for the Generator 
    #activation_continuous: the activation function used in the output layers connected to continuous variables
    #activation_nominal: the activation function used in the output layers connected to nominal variables
    ## Experimental Design  WGANGP Paper
    '''
    
    ## Main model
    inputs= keras.Input(shape=latent_dim)
    X= layers.Dense(256, activation= ReLU())(inputs)
    X= BatchNormalization()(X)
    X= layers.Dense(256, activation= ReLU())(X)
    X= BatchNormalization()(X)
    outputs= layers.Dense(output_shape)(X)
    
    ## Create Dense layers for every column of the dataset  
    continuousColumns= layers.Dense(14, name="continuousDense")(outputs)
    education= layers.Dense(7, name='educationDense')(outputs)
    marriage= layers.Dense(4, name='marriageDense')(outputs)
    pay_1= layers.Dense(11, name='pay1Dense')(outputs)
    pay_2= layers.Dense(10, name='pay2Dense')(outputs)
    pay_3= layers.Dense(11, name='pay3Dense')(outputs)
    pay_4= layers.Dense(11, name='pay4Dense')(outputs)
    pay_5= layers.Dense(10, name='pay5Dense')(outputs)
    pay_6= layers.Dense(10, name='pay6Dense')(outputs)
    sex= layers.Dense(2, name='sexDense')(outputs)
    default_payment_next_month= layers.Dense(2, name='defaultDense')(outputs)
    
    ## Create new output layers for every column of the dataset 
    continuousColumns= layers.Dense(14, activation= activation_continous, name="continuousOutput")(continuousColumns)
    education= layers.Dense(7, activation= activation_discrete, name='educationOutput')(education)
    marriage= layers.Dense(4, activation= activation_discrete, name='marriageOutput')(marriage)
    pay_1= layers.Dense(11, activation= activation_discrete, name='pay1Output')(pay_1)
    pay_2= layers.Dense(10, activation= activation_discrete, name='pay2Output')(pay_2)
    pay_3= layers.Dense(11, activation= activation_discrete, name='pay3Output')(pay_3)
    pay_4= layers.Dense(11, activation= activation_discrete, name='pay4Output')(pay_4)
    pay_5= layers.Dense(10, activation= activation_discrete, name='pay5Output')(pay_5)
    pay_6= layers.Dense(10, activation= activation_discrete, name='pay6Output')(pay_6)
    sex= layers.Dense(2, activation= activation_discrete, name='sexOutput')(sex)
    default_payment_next_month= layers.Dense(2, activation= activation_discrete, name='defaultOutput')(default_payment_next_month)
    
    
    ## Concatenate all output layers together again 
    lastLayer= layers.concatenate([continuousColumns, education, marriage, pay_1, pay_2, pay_3,
                                  pay_4, pay_5, pay_6, sex, default_payment_next_month])
    modelFinal= keras.Model(inputs=inputs, outputs=lastLayer)
    
    # no need to compile (will be done within WGAN class [see below])    
    return modelFinal


In [ ]:
def generator_setup_WGANGP_GENERIC(output_shape, latent_dim, numberOfNodes, numberOfLayers,  
                                        nominalColumnsValues, nominalColumns, continuousColumns_dim,
                                        activation_continous='tanh', activation_discrete='softmax'):
    '''
    #output_shape: the dimension of the dataset we need to generate. 
    #laten_dim: the dimension of the latent space where we draw our input for the Generator 
    #activation_continuous: the activation function used in the output layers connected to continuous variables
    #activation_nominal: the activation function used in the output layers connected to nominal variables
    ## Experimental Design  WGANGP Paper -> works as expected! 
    '''
    
    ## Main model
    inputs= keras.Input(shape=latent_dim)
    X= inputs
    for eachLayer in range(numberOfLayers):
        X= layers.Dense(numberOfNodes, activation= ReLU())(X)
        X= BatchNormalization()(X)
    outputs= layers.Dense(output_shape)(X)
    
    ## Create Dense layers for every column of the dataset  
    continuousColumns= layers.Dense(continuousColumns_dim, name="continuousDense")(outputs)
    nominalColumnLayers= [layers.Dense(eachColumn, name= eachName)(outputs) for (eachColumn, eachName) 
                     in zip(nominalColumnsValues, nominalColumns)]
    
    ## Create new output layers for every column of the dataset 
    nominalColumnsNames2 = [eachName + '_Output' for eachName in nominalColumns]
    continuousColumns= layers.Dense(continuousColumns_dim, activation= activation_continous, name="continuousOutput")(continuousColumns)
    nominalColumns= [layers.Dense(eachColumn, activation= activation_discrete, name= eachName)(eachLayer) 
                     for (eachColumn, eachName, eachLayer) 
                     in zip(nominalColumnsValues, nominalColumnsNames2, nominalColumnLayers)]
    
    ## Concatenate all output layers together again 
    lastLayer= layers.concatenate([continuousColumns, *nominalColumns])
    modelFinal= keras.Model(inputs=inputs, outputs=lastLayer)
    
    # no need to compile (will be done within WGAN class [see below])    
    return modelFinal


# GAN MODEL: Composite model of Discriminator and Generator 

-----------


## GAN 
--------------
#### Define logical third-model that combines the generator and discriminator
- sidenote line 4: Making the discriminator not trainable is a clever trick in the Keras API. The trainable property impacts the model after it is compiled. The discriminator model was compiled with trainable layers, therefore the model weights in those layers will be updated when the standalone model is updated via calls to the train_on_batch() function.

In [ ]:
def gan_setup(generator, discriminator):
    '''
    #Generator: NN Generator 
    #Discriminator: NN Discriminator
    ## Basic GAN-model 
    '''
    
    # make weights in the discriminator not trainable from POV of GAN-model
    discriminator.trainable = False
    # connect models 
    model = Sequential()
    # add generator 
    model.add(generator)
    # add discriminator
    model.add(discriminator)
    
    # compile model 
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

## WGAN-GP
-----------
#### Use Subclassing API to override default Model.train_step() from keras library to incorporate Gradient-Penalty from WGAN-GP

In [ ]:
class WGAN(keras.Model):
    '''
    Improved Wasserstein GAN model with Gradient Penalty.
    Subclassed from the keras.Model Class
    --------
    Paper: Wasserstein GAN, Martin Arjovsky, Soumith Chintala, and L´eon Bottou. Courant Institute of Mathematical Sciences,
    Facebook AI Research

    '''
    
    def __init__(self, discriminator, generator, latent_dim, discriminator_extra_steps= 5, gp_weight= 10.0,):
        # Call the parent constructor
        super(WGAN, self).__init__()
        # Initialize the individiual components of the model and parameters from the paper  
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_steps = discriminator_extra_steps
        self.gp_weight = gp_weight
    
    # We need two optimizers and two loss functions (discriminator and generator)
    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(WGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn
    
    # ----------------
    # Gradient Penalty
    # ----------------
    
    def gradient_penalty(self, batch_size, Xreal, Xfake):
        """ Computes the gradient penalty.

        Computes the gradient penalty of Eq.3 on a random interpolated dataset constructed 
        of the real and artificial datasets.
        """
        # Compute random weighted average 
        alpha = tf.random.normal([batch_size, 1], 0.0, 1.0)
        diff = Xfake - Xreal
        interpolated = Xreal + alpha * diff
        
        # Compute the Gradient Penalty 
        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Run Discriminator on interpolated dataset 
            pred = self.discriminator(interpolated, training=True)

        # 2. Compute gradients wrt to the interpolated dataset 
        grads = gp_tape.gradient(pred, [interpolated])[0]
        # 3. Compute the norm of the gradient
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis= 1))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp
    
    
    # -----------------------
    # Modified train_step() -> will be called during every batch update during self.fit() 
    # ------------------------
    

    def train_step(self, Xreal):
        
        # When calling mdl.fit(x=None, y=None, ...) x and y get converted into the tuple (x,y)
        #so we need to separate them again 
        if isinstance(Xreal, tuple):
            Xreal = Xreal[0]

        # Extract batch size
        batch_size = tf.shape(Xreal)[0]

        ## Algorithm 1 of original paper 
        
        #1. train generator and get the loss 
        #2. train discriminator and get the loss
        #3. compute gradient penalty
        #4. multiply gradient penalty with lambda 
        #5. add new gradient penalty to discriminator loss (to get eq.3)
        #6. save generator and discriminator losses 

        # DISCRIMINATOR TRAINING (trained more freq. than generator)
        
        for i in range(self.d_steps):
            
            # Get the latent vector
            random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
            # Gradient penalty
            with tf.GradientTape() as tape:
                #draw samples from generator distribution 
                Xfake = self.generator(random_latent_vectors, training=True)
                #train discriminator on the fake and real samples respectively
                fake_logits = self.discriminator(Xfake, training=True)
                real_logits = self.discriminator(Xreal, training=True)
                # Calculate discriminator loss using fake and real logits
                d_cost = self.d_loss_fn(Xreal=real_logits, Xfake=fake_logits)
                # Calculate the gradient penalty
                gp = self.gradient_penalty(batch_size, Xreal, Xfake)
                # Add the gradient penalty to the original discriminator loss
                d_loss = d_cost + gp * self.gp_weight

            # Get the gradients w.r.t the discriminator loss
            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            # Update the weights of the discriminator using the discriminator optimizer
            self.d_optimizer.apply_gradients(zip(d_gradient, self.discriminator.trainable_variables))

        # GENERATOR TRAINING
        
        # Get the latent vector
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        with tf.GradientTape() as tape:
            #draw samples from generator distribution 
            Xfake_generator = self.generator(random_latent_vectors, training=True)
            #evaluate samples with the discriminator 
            gen_Xfake_logits = self.discriminator(Xfake_generator, training=True)
            #compute generator loss 
            g_loss = self.g_loss_fn(gen_Xfake_logits)

        # Get the gradients w.r.t the generator loss
        g_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(g_gradient, self.generator.trainable_variables))
        
        return {"d_loss": d_loss, "g_loss": g_loss }

#### Create custom callback to evaluate the KL-divergence after each epoch 
- See KLDivergence() function for more information about the KL estimator 

In [ ]:
class GANMonitor(keras.callbacks.Callback):
    '''
    Input
    ---------
    Xreal: sample of the true distribution 
    sample_size: size of the sample (the greater the size the slower the computation)
    latent_dim= dimension of latent space to draw from
    k: the kth nearest neighbor to use for the density estimation of the KL estimator
    --------
    Output
    ---------- 
    kl_tracker: keeps track of the KL value after every epoch 
    ----------------
    Custom callback that allows us to estimate the KL-divergence between the true and generated distribution 
    after every epoch when model.fit() is called. 
    --------------
    see: https://keras.io/guides/customizing_what_happens_in_fit/ 
    
    '''
    
    def __init__(self, Xreal, sample_size, latent_dim, k):
        self.Xreal= Xreal
        self.sample_size= sample_size
        self.latent_dim= latent_dim
        self.k= k
        self.kl_tracker= []
    
    def on_epoch_end(self, epoch, logs=None):
        
        # Generate a set of real samples 
        Xreal, _= generate_real_samples(self.Xreal, self.sample_size)
        
        # Generate a set of fake samples from the semi-trained Generated 
        Xfake, _= generate_artificial_samples(self.model.generator, self.latent_dim, self.sample_size)
        
        # Compute KL-divergence with our KL-KNN estimator (see KLdivergence function for more details)
        kl= KLdivergence(Xreal, Xfake, k= self.k, dim= 2)
        self.kl_tracker.append(kl)
    

# Training the GAN Model 
--------------------------------------------
## Draw random samples from the real dataset 


In [ ]:
def generate_real_samples(dataset, n_samples):
    '''
    #dataset: dataset to draw from 
    #n_samples: number of samples to draw 
    ## function selects random real samples to feed in batches to the discriminator (works better with the stoch grad descent)
    '''
    # choose random instances
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    # retrieve selected datapoints 
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = np.ones((n_samples, 1))
    return X, y

## Draw random samples from the Generator's latent space 
---------------

In [ ]:
def generate_latent_points(latent_dim, n_samples):
    '''
    #latent_dim: dimension of latent space 
    #n_samples: number of samples to draw 
    ## generate points in the latent space
    '''
    x_input = np.random.randn(latent_dim * n_samples) #generate x_input~N(0,1) for each sample 
    # reshape into a batch of inputs for the network 
    x_input = x_input.reshape(n_samples, latent_dim) #noise input dataset with n_samples and latent_dim features 
    return x_input

## Generate samples from the Generator
------------------

In [ ]:
def generate_artificial_samples(model, latent_dim, n_samples):
    '''
    #model: the Generator model to draw from 
    #latent_dim: dimension of latent space 
    #n_samples: number of samples to draw 
    ## generate artificial samples made by the Generator
    '''
    #generate samples from the latent space 
    gen_input= generate_latent_points(latent_dim, n_samples)
    #convert samples to our outputspace with the Generator
    X= model.predict(gen_input)
    #create class 'fake' labels (0)
    y = np.zeros((n_samples, 1))
    return X, y

## Save plot of GAN output to evaluate between epochs
-----------
- Auxiliary function to performance() to visually inspect some generated data samples. 
- At the moment, it makes only sense to run if data is actually respresenting something visually (e.g. An actual image; A yield Curve; Credit spread, etc.) 

In [ ]:
def save_plot(examples, epoch, n=10, output_directory= 'GColab'): 
    '''
    #examples: samples of the generated data 
    #epoch: the current epoch 
    #n = sqrt() of number of examples to save (to create a square grid)
    #output_directory= 'GColab' if we're running on Google Colab, 'desktop' if we're running from JNotebook 
    '''
    #plot 
    for sample in range(n**2): 
        plt.subplot(n, n, sample+1)
        plt.axis('off')
        plt.plot(examples[sample, :])
    
    #Save plot
    if output_directory == 'desktop':
        #save plot to file
        filename = 'generated_plot_e%03d.png' % (epoch+1)
        plt.savefig(filename)
        plt.close()
        
    if output_directory == 'GColab':
        #save plot to file
        filename = '/content/gdrive/My Drive/Stress Testing with GANs/Images/generated_plot_e%03d.png' % (epoch+1)
        plt.savefig(filename)    
        plt.close()

## Keep track of performance during training 
-----------
#### Evaluate the performance of the discriminator based on accuracy on the 'real' and 'fake' samples  and save the corresponding generator model 
- The function will be called during training between running epochs so we ex-post evaluate when the optimal stopping epoch is reached
- Only works with Vanilla GAN setup [i.e. gan_setup()]
- WGAN-GP is subclassed from the keras.Model class so we need a different way of tracking performance -> See custom callback function 


In [ ]:
def performance(epoch, g_mdl, d_mdl, dataset, latent_dim, n_samples= 100, output_directory = 'GColab'):
    '''
    #epoch: the current epoch 
    #g_mdl: generator 
    #d_mdl: discriminator 
    #dataset: the original dataset 
    #latent_dim: dimension of latent space
    #n_samples: number of samples to draw from the real and artificial dataset 
    #output_directory= 'GColab' if we're running on Google Colab, 'desktop' if we're running from JNotebook 
    ## Returns accuracy of Discriminator over real and fake samples (in tuple form)
    '''
    
    #generate 'real' samples
    Xreal, Yreal = generate_real_samples(dataset, n_samples)
    #evaluate discriminator on 'real' samples 
    _, acc_real = d_mdl.evaluate(Xreal, Yreal, verbose= 0)
    
    #generate 'fake' samples
    Xfake, Yfake = generate_artificial_samples(g_mdl, latent_dim, n_samples)
    #evaluate discriminator on 'fake' samples 
    _, acc_fake = d_mdl.evaluate(Xfake, Yfake, verbose= 0)
    
    #print performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    
    #save Generator 
    
    if output_directory == 'desktop':
        filename = 'generator_model_%03d.h5' % (epoch + 1)
        g_mdl.save(filename)
    
    #save Generator GColab ####
    if output_directory == 'GColab':
        filename = '/content/gdrive/My Drive/Stress Testing with GANs/Images/generator_model_%03d.h5' % (epoch+1)
        g_mdl.save(filename)
    
    #save plot
    save_plot(Xfake, epoch, 10, output_directory)
    
    return acc_real, acc_fake

## Track performance visually 
-----------
- Auxiliary function to train_gan() to track performance during training phase. 
- Only works with Vanilla GAN setup [i.e. gan_setup()]
- WGAN-GP is subclassed from the keras.Model class so we need a different way of tracking performance -> see plot_metrics() function 

In [ ]:
def plot_loss(d_loss, g_loss, d_real_acc, d_fake_acc):
    fig= plt.figure(figsize=[10,5])
    # Loss plots
    plt.subplot(2,1,1)
    plt.plot(d_loss, label= 'Discriminator')
    plt.plot(g_loss, label= 'Generator')
    plt.legend()
    plt.title('Loss Functions during Training')
    
    # Accuracy plots
    plt.subplot(2,1,2)
    plt.plot(d_real_acc, label= 'Real samples')
    plt.plot(d_fake_acc, label= 'Fake samples')
    plt.legend()
    plt.title('Accuracy of the Discriminator During Training')
    plt.tight_layout()

### WGAN-GP version to track performance


In [ ]:
# Plots the loss functions of Discriminator and Generator and the KL-divergence obtained during training 
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

## Main function

def plot_metrics(history, cbk, zoom_in= True):
  '''
  INPUT
  --------
  history: Dictionary containing losses obtained after running wgan.fit()
  cbk: callback obtained after running wgn.fit()
  
  OUTPUT
  ---------
  plots the losses and Kullback-Leibler divergence during training
  '''

  g_loss= history.history['g_loss']
  d_loss= history.history['d_loss']
  kl= np.array(cbk.kl_tracker)

  #function to set our spines invisible 
  def make_patch_spines_invisible(ax):
      ax.set_frame_on(True)
      ax.patch.set_visible(False)
      for sp in ax.spines.values():
          sp.set_visible(False)



  # ------------------        
  ## Main Graph 
  # ------------------


  #setup figure 
  fig, host = plt.subplots(figsize=[20, 5])
  fig.subplots_adjust(right=0.75)
  par1 = host.twinx()
  par2 = host.twinx()
  # Offset the right spine of par2.  The ticks and label have already been
  # placed on the right by twinx above.
  par2.spines["right"].set_position(("axes", 1.1))
  # Having been created by twinx, par2 has its frame off, so the line of its
  # detached spine is invisible.  First, activate the frame but make the patch
  # and spines invisible.
  make_patch_spines_invisible(par2)
  # Second, show the right spine.
  par2.spines["right"].set_visible(True)
  # Ready to plot our graphs 
  epochs = range(1, len(g_loss) + 1)
  p1, = host.plot(epochs, kl, 'b', label="KL-Divergence")
  p2, = par1.plot(d_loss, 'r', label="d_loss")
  p3, = par2.plot(g_loss, 'g', label="g_loss")
  # Name labels appropriately 
  host.set_xlabel("Epochs", fontweight='bold')
  host.set_ylabel("KL-Divergence", fontweight='bold')
  par1.set_ylabel("d_loss", fontweight='bold')
  par2.set_ylabel("g_loss", fontweight='bold')
  # Keep coloring uniform across the entire figure 
  host.yaxis.label.set_color(p1.get_color())
  par1.yaxis.label.set_color(p2.get_color())
  par2.yaxis.label.set_color(p3.get_color())
  # Clean up ticks 
  tkw = dict(size=4, width=1.5)
  host.tick_params(axis='y', colors=p1.get_color(), **tkw)
  par1.tick_params(axis='y', colors=p2.get_color(), **tkw)
  par2.tick_params(axis='y', colors=p3.get_color(), **tkw)
  host.tick_params(axis='x', **tkw)
  # Add outside legend and give title
  lines = [p1, p2, p3]
  host.legend(lines, [l.get_label() for l in lines], title= 'LEGEND', bbox_to_anchor=(1.30, 1),)
  plt.title('Generator/Discriminator Loss functions and the KL-divergence', fontweight='bold')


  # ------------------------------
  ## Zoom-ins to get more detail 
  # ------------------------------

  if zoom_in == True:

    # Zoom into a part of KL-divergence graph
    axins = zoomed_inset_axes(host, 2, loc= 5) # (graph, zoom-in factor, location on graph to display the zoom)
    #What to zoom in on
    axins.plot(epochs, kl)
    #limits 
    x1= len(kl) - int(np.round(0.1*len(kl)))
    x2= len(kl)
    y1= np.min(kl[x1: x2])
    y2= np.max(kl[x1: x2])
    axins.set_xlim(x1, x2) # apply the x-limits
    axins.set_ylim(y1, y2) # apply the y-limits
    #Add connecting lines to original plot 
    mark_inset(host, axins, loc1=3, loc2=4, fc="none", ec="0", )

    #Zoom into a part of the Discriminator loss graph
    axins = zoomed_inset_axes(par1, 2, loc= 10) # (graph, zoom-in factor, location on graph to display the zoom)
    #What to zoom in on
    axins.plot(epochs, d_loss, 'r')
    #limits 
    x1= len(d_loss) - int(np.round(0.1*len(d_loss)))
    x2= len(d_loss)
    y1= np.min(d_loss[x1: x2])
    y2= np.max(d_loss[x1: x2])
    axins.set_xlim(x1, x2) # apply the x-limits
    axins.set_ylim(y1, y2) # apply the y-limits
    #Add connecting lines to original plot 
    mark_inset(par1, axins, loc1=2, loc2=4, fc="None", ec="0", )

  plt.show()


## Training the GAN Model 
-------------------


In [ ]:
def train_gan(d_mdl, g_mdl, gan_mdl, dataset, latent_dim, n_epochs= 100, batch_size= 10000
          , plotLoss= False, output_directory= 'GColab'):
    '''
    #d_mdl: discriminator 
    #g_mdl: generator 
    #gan_mdl: composite model of discriminator and generator 
    #latent_dim: dimension of latent space
    #dataset: the original dataset 
    #n_epochs: number of epochs (which determines the number of updates to the generator)
    #batch_size: batch size (which determines the number of updates to the discriminator)
    '''
    
    numberOfBatches = int(dataset.shape[0] / batch_size) #number of batches in one epoch 
    half_batch = int(batch_size / 2)
    #initialize lists to track performance over training batches
    d_loss_track, g_loss_track, d_real_acc, d_fake_acc= list(), list(), list(), list() 
    #loop through epochs
    for epoch in range(n_epochs):
        #loop through batches:
        for batch in range(numberOfBatches):
            
            # ----------------
            # DISCRIMINATOR
            # ----------------
            
            #generate random 'real' samples
            Xreal, Yreal= generate_real_samples(dataset= dataset, n_samples= half_batch)
            #generate random 'fake' samples 
            Xfake, Yfake= generate_artificial_samples(g_mdl,latent_dim, half_batch)
            #combine 'real' and 'fake' samples into one training set for the discriminator to train on 
            X, y = np.vstack((Xreal, Xfake)), np.vstack((Yreal, Yfake))
            #update discriminator 
            d_loss, d_acc = d_mdl.train_on_batch(X, y)
            #track loss function
            d_loss_track.append(d_loss)
            
            # --------------
            # GENERATOR 
            # --------------
            
            #generate latent space 
            Xgan= generate_latent_points(latent_dim, batch_size)
            #create INVERTED labels (0 -> 1)
            yGan= np.ones((batch_size, 1))
            #update generator (based on discriminator errors)
            g_loss = gan_mdl.train_on_batch(Xgan, yGan)
            #track loss function
            g_loss_track.append(g_loss)
            #summarize performance on this batch 
            print("LOSS:")
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (epoch+1, batch+1, numberOfBatches, d_loss, g_loss))
            print("----------------------")
            
        # evaluate the model performance, sometimes
        if (epoch+1) % 1 == 0:
            #print("LOSS:")
            #print('>%d, %d/%d, d=%.3f, g=%.3f' % (epoch+1, batch+1, numberOfBatches, d_loss, g_loss))
            #print("----------------------")
            performance(epoch, g_mdl, d_mdl, dataset, latent_dim, 100, output_directory)
            
            # Track accuracy of Discriminator 
            #generate 'real' samples
            Xreal, Yreal = generate_real_samples(dataset, 1000)
            #evaluate discriminator on 'real' samples 
            _, acc_real = d_mdl.evaluate(Xreal, Yreal, verbose= 0)
            #generate 'fake' samples
            Xfake, Yfake = generate_artificial_samples(g_mdl, latent_dim, 1000)
            #evaluate discriminator on 'fake' samples 
            _, acc_fake = d_mdl.evaluate(Xfake, Yfake, verbose= 0)
            # save performance
            d_real_acc.append(acc_real)
            d_fake_acc.append(acc_fake)
            
    if plotLoss == True:
        plot_loss(d_loss_track, g_loss_track, d_real_acc, d_fake_acc)
            

## Training the WGAN-GP Model 
-------------------

In [ ]:
def compile_wgan(dataset_dim, dataset_nominal_dim, dataset_continuous_dim, nominalColumns,
                 epochs= 1000, noise_dim= 512, batch_size= 512, 
                 discriminator_steps= 5, sample_size_kl= 1000, knn= 10):

  ## Optimizer for both the networks
  #learning_rate=0.0002, beta_1=0.5 are recommended (keep hardcoded for now)
  generator_optimizer = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.9)
  discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.9)


  ## Define the loss function to be used for the Discrimiator
  # Gradient Penalty is added when calling WGAN.fit()
  def discriminator_loss(Xreal, Xfake):
      real_loss = tf.reduce_mean(Xreal)
      fake_loss = tf.reduce_mean(Xfake)
      return fake_loss - real_loss

  ## Define the loss functions to be used for the Generator
  def generator_loss(Xfake):
      return -tf.reduce_mean(Xfake)

  ## MODEL 
  d_model = discriminator_setup_WGANGP(dataset_dim)
  g_model = generator_setup_WGANGP_GENERIC(output_shape= dataset_dim, latent_dim= noise_dim, 
                                           numberOfNodes= 256, numberOfLayers= 2, nominalColumnsValues= dataset_nominal_dim, 
                                           nominalColumns= nominalColumns, continuousColumns_dim= dataset_continuous_dim,
                                           activation_continous='tanh', activation_discrete='softmax')
  # Get the wgan model
  wgan = WGAN(discriminator=d_model, generator=g_model, latent_dim=noise_dim, discriminator_extra_steps= discriminator_steps)
  # Compile the wgan model
  wgan.compile(d_optimizer=discriminator_optimizer, g_optimizer=generator_optimizer, g_loss_fn=generator_loss, d_loss_fn=discriminator_loss)

  return d_model, g_model, wgan

# EVALUATING THE MODEL 
----------------
## Kullback Leibler Divergence 

In [ ]:
def KLdivergence(Xreal, Xfake, k=1, dim= 2):
    """
    Kullback-Leibler estimator in a multivariate universe.

    #x : dataset sampled from distribution P()
    #y : dataset sampled from distribution Q() [from the same multivariate universe]
    #dim: number of dimensions 
    #output : Estimate of D(P||Q) based on KNN density estimates

    #see: 
    - Pérez-Cruz, F. Kullback-Leibler divergence estimation of
    continuous distributions
    - Qing Wang, Sanjeev R. Kulkarni, Sergio Verd´u, 
    A Nearest-Neighbor Approach to Estimating Divergence between Continuous Random Vectors
    """
    
    # Parameters 
    
    #add second dimension in case of vectors 
    if dim == 1:
        Xreal= Xreal.reshape(-1,1)
        Xfake= Xfake.reshape(-1,1)
        
    n,d = Xreal.shape
    m,dy = Xfake.shape
    
    # K Nearest Neighbors [KD tree to speed up process, but results in possible mistakes (!)]
    XrealTree = cKDTree(Xreal)
    Xfaketree = cKDTree(Xfake)

    # Get (euclidian) distance to closest neighbors (k= 1)
    if k == 1:
        r = XrealTree.query(Xreal, k=2)[0][:,1] #1st "neighbor" is x itself, so we look at 2nd neighbor
        s = Xfaketree.query(Xreal, k=1)[0]
    else:
        r = XrealTree.query(Xreal, k= k+1)[0][:,-1] #1st "neighbor" is x itself, so we look at 2nd neighbor
        s = Xfaketree.query(Xreal, k= k)[0][:,-1]
    
    # Addendum Paper Pérez-Cruz et al. Eq.14: add negative sign to first right-hand side term (see Qing Wang et al.)
    return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))

## Test for Normality (To be used when original dataset is drawn from N~(mu, cov) )
--------------

In [ ]:
def Normality(Xreal, Xfake, multivariate_test= 'False', plot= True, dim_plot= -1): 
    '''
    #Xreal: samples from the real dataset 
    #Xfake: samples from the artificial dataset
    #multivariate_test: Henze-Zirkler multivariate normality test of the pingouin library 
    (test requires sign. memory allocation!)
    #plot: plots the distributions of the real and generated dataset 
    #dim_plot: which dimension to plot 
    ## Compare distribution structures between the generated data and the real data 
    '''
    from scipy.stats import normaltest, shapiro
    from scipy.stats import skew, kurtosis
    
    # Compare mean vector 
    mean_real = np.round(np.mean(Xreal, axis= 0), 2)
    mean_fake = np.round(np.mean(Xfake, axis= 0), 2)
    # Compare first Covariance matrix 
    corr_real= np.round(np.corrcoef(Xreal, rowvar= False), 2)
    corr_fake= np.round(np.corrcoef(Xfake, rowvar= False), 2)
    corr_diff= corr_real - corr_fake
    # Compare Skewness 
    skew_real= np.round(skew(Xreal, axis= 0), 2)
    skew_fake= np.round(skew(Xfake, axis= 0), 2)
    # Compare Kurtosis 
    kurtosis_real= np.round(kurtosis(Xreal, axis= 0), 2)
    kurtosis_fake= np.round(kurtosis(Xfake, axis= 0), 2)
    print('REAl: mu', mean_real)
    print('FAKE: mu', mean_fake)
    print('--------')
    print('REAl: cov', corr_real)
    print('FAKE: cov', corr_fake)
    print('Difference: cov', corr_diff)
    print('--------')
    print('REAl: skew', skew_real)
    print('FAKE: skew', skew_fake)
    print('--------')
    print('REAl: kurt', kurtosis_real)
    print('FAKE: kurt', kurtosis_fake)
    print('--------')
    
    # Test for normalitiy 
    if multivariate_test == True:
        from pingouin import multivariate_normality
        multivariate_normality(dataset_Gaussian)
    # Plot distribution (from last variable)
    if plot == True:
        fig = plt.figure(figsize=[10,5])
        plt.hist(Xreal[:, dim_plot], bins= 100, label= 'Real Distribution');
        plt.hist(Xfake[:, dim_plot], bins= 100, label= 'Generated Distribution');
        plt.legend()
        plt.title('Real vs. Generated Distribution')        

# Processing the data 
--------------------
- Functions used to pre- and postprocess the data 

In [ ]:
# Transforms ohe dataset to list of individual ohe variables 
def variableSeparator(nominalColumnsValues, nominalDataset):
    '''
    INPUT
    ------------
    nominalColumnsValues: A list containing the number of unique values of each variable 
    nominalDataset: a dataset containing one-hot encoded nominal variables 
    ---------------
    OUTPUT
    ----------------
    list with the i-th element containing all of the one-hot encoded columns of the i-th nominal variable  
    ----------------
    Separates all of the individual nominal variables into a list, so it's easy to perform computations on each variable
    '''
    #initialize loop
    dim= len(nominalColumnsValues)
    variablesSeparated= []
    #loop through each variable 
    for eachVariable in range(dim):
        #special case when starting off the loop where we can't use the idx variable yet
        if eachVariable == 0:
            tmp= nominalDataset[:, eachVariable:eachVariable+nominalColumnsValues[eachVariable]]
            variablesSeparated.append(tmp)
            idx= eachVariable+nominalColumnsValues[eachVariable]
        #extract the all of one-hot encoded columns attached to the individual variable 
        else: 
            tmp= nominalDataset[:, idx:idx+nominalColumnsValues[eachVariable]]
            variablesSeparated.append(tmp)
            idx+= nominalColumnsValues[eachVariable]
            
    return variablesSeparated

In [ ]:
# Tranforms probabilities to ohe format (used in postprocessing)
def retransformer(nominalVariable): 
  '''
  INPUT
  -----------
  nominalVariable: probability matrix linked to the ohe matrix of a nomnial variable
  ------------
  OUTPUT
  ----------
  ohe matrix of the nominal variable
  -------------------------
  Function used in the postprocessing phase to go back from ohe probabilities to simple ohe.  

  '''
  idx = nominalVariable.argmax(axis=1)
  out = np.zeros_like(nominalVariable,dtype=float)
  out[np.arange(nominalVariable.shape[0]), idx] = 1
  return out